In [1]:
import fastText
import math
import linecache
import numpy as np 
from numpy import random
from random import sample
from keras.models import Sequential, Model
from keras.callbacks import ModelCheckpoint
from keras.layers import *
from keras import *
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras.initializers import RandomUniform
import re
from sklearn.metrics import f1_score, recall_score, precision_score, accuracy_score

# from attention_utils import get_activations, get_data_recurrent


/home/jindal/miniconda3/envs/NER2/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=0

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=0


In [3]:
ft = fastText.load_model("/home/jindal/notebooks/fastText/wiki.de.bin")

nb_embedding_dims = ft.get_dimension()
nb_sequence_length = 75

In [4]:
def twitter_tokenizer(textline):
    textLine = re.sub(r'http\S+', 'URL', textline)
    textline = re.sub('@[\w_]+', 'USER_MENTION', textline)
    textline = re.sub('\|LBR\|', '', textline)
    textline = re.sub('\.\.\.+', '...', textline)
    textline = re.sub('!!+', '!!', textline)
    textline = re.sub('\?\?+', '??', textline)
    words = re.compile('[\U00010000-\U0010ffff]|[\w-]+|[^ \w\U00010000-\U0010ffff]+', re.UNICODE).findall(textline.strip())
    words = [w.strip() for w in words if w.strip() != '']
    # print(words)
    return(words)

In [5]:
word_vectors_ft = {}
def process_features(textline, nb_sequence_length, nb_embedding_dims, tokenize=True):
    if not tokenize:
        words = textline.split()
    else:
        words = twitter_tokenizer(textline)
    features_ft = np.zeros((nb_sequence_length, nb_embedding_dims))
    features_idx = np.zeros(nb_sequence_length)
    max_words = min(len(words), nb_sequence_length)
    idx = nb_sequence_length - len(words[:max_words])
    for w in words[:max_words]:
        if w in word_vectors_ft:
            wv = word_vectors_ft[w]
        else:
            wv = ft.get_word_vector(w.lower())
            word_vectors_ft[w] = wv
        features_ft[idx] = wv
        
        idx = idx + 1
    return features_ft

In [6]:
def sequential_generator(filename, 
                         batch_size, 
                         labels2Idx:'dict to make output labels',
                         check:'to check if all lines in file are of same length.To check enter the len of line after splitting it by tabs' = None,
                         tokenize:'specify if using twitter tokenzor to preprocess lines'=False, 
                        ):    
    
    f = open(filename)
    while True:
        batch_features_ft = np.zeros((batch_size, nb_sequence_length, nb_embedding_dims))
#         print(type(labels2Idx))
        batch_labels = np.zeros((batch_size, len(labels2Idx)))
        for i in range(batch_size):
            line = f.readline()
            if ("" == line):
                f.seek(0)
                line = f.readline()
            data = line.strip().split('\t')
            if check:
                if len(data)!=check:
                    i-=1
                    continue
            batch_features_ft[i] = process_features(data[0], nb_sequence_length, nb_embedding_dims, tokenize= tokenize)
            if len(labels2Idx)==2:
                batch_labels[i] = to_categorical(0 if data[1] == 'OTHER' else 1, n_labels)
            else:
                batch_labels[i] = to_categorical(labels2Idx[data[1]], n_labels)
        yield ([batch_features_ft], batch_labels)

In [7]:
def train_dev_sentences(filetrain,filedev, check:'to check if lines of file are all same lenght after separating by tab'):
    labels2Idx = {}
    train_lines = [line.strip().split("\t") for line in open(filetrain) if len(line.strip().split('\t'))==check]
    dev_lines = [line.strip().split("\t") for line in open(filedev) if len(line.strip().split('\t'))==check]

    train_sentences = [x[0] for x in train_lines]
    for dataset in [train_lines, dev_lines]:
        for line in dataset:
            label = line[1]
            if label not in labels2Idx.keys():
                labels2Idx[label]= len(labels2Idx)
                
#     train_labels = [0 if x[1] == "OTHER" else 1 for x in train_lines]
    train_labels = [labels2Idx[x[1]] for x in train_lines]
    dev_sentences = [x[0] for x in dev_lines]
#     dev_labels = [0 if x[1] == "OTHER" else 1 for x in dev_lines]
    dev_labels = [labels2Idx[x[1]] for x in dev_lines]
    return (train_sentences, train_labels, dev_sentences, dev_labels, labels2Idx)


In [8]:
file_train = '/home/jindal/notebooks/jindal/NER/language_model/twitter_emoji_train.csv'
file_dev = '/home/jindal/notebooks/jindal/NER/language_model/twitter_emoji_dev.csv'
train_sentences, train_labels, dev_sentences, dev_labels, labels2Id = train_dev_sentences(filetrain=file_train,filedev=file_dev, check=2)

In [9]:
print(len(dev_sentences))

34271


In [10]:
dev_labels[:10]

[5, 11, 5, 9, 14, 2, 10, 14, 10, 0]

In [11]:
print(len(dev_labels))

34271


In [12]:
n_labels = len(labels2Id)
print(n_labels)

39


In [13]:
def compile_model(no_labels:'total labels for classification'):
    model_input_embedding = Input(shape = (nb_sequence_length, nb_embedding_dims))
    lstm_block = Bidirectional(LSTM(100, dropout = 0.5, return_sequences=True))(model_input_embedding)
    lstm_block = LeakyReLU()(lstm_block)

    filter_sizes = (3, 4, 5)
    conv_blocks = []
    for sz in filter_sizes:
        conv = Conv1D(
            filters = 200,
            kernel_size = sz,
            padding = 'valid',
            strides = 1
        )(lstm_block)
        conv = LeakyReLU()(conv)
        conv = GlobalMaxPooling1D()(conv)
        conv = Dropout(0.5)(conv)
        conv_blocks.append(conv)
    model_concatenated = concatenate([conv_blocks[0], conv_blocks[1], conv_blocks[2]])
    # model_concatenated = Dropout(0.8)(model_concatenated)
    model_concatenated = Dense(100)(model_concatenated)
    model_concatenated = LeakyReLU()(model_concatenated)
    model_output = Dense(no_labels, activation = "softmax")(model_concatenated)
    new_model = Model(model_input_embedding, model_output)
    new_model.compile(loss='categorical_crossentropy', optimizer='nadam', metrics = ['accuracy'])
#     new_model.summary()
    return new_model

In [14]:
print(dev_sentences[1])

happy birthdayXx USERMENTION


In [15]:
def test_model(generator, 
               train_sentences, 
               devLabels, 
               number_of_tests,
               number_of_epochs,
               filename_to_log, 
               labels_earlier:'number of original labels if loading a pretrained model',
               filename_to_save_weigths,
               batch_size, 
               train_file:'filepath for traininig',
               f1_measure:'binary/macro etc', 
               pos_label:'only if binary f1',
               labels2Idx,
               load_model_weights=False,
               model_weights_file:'give filepath as str'=None, 
               tokenize=True,
               nb_sequence_length = nb_sequence_length,
               nb_embedding_dims= nb_embedding_dims, 
               check_for_generator=None,
                ):
    
#     f = open(filename_to_log,"w")
    
    max_f1=0
    max_p=0
    max_r=0
    max_a=0
    total_f1=0
    total_prec=0
    total_acc=0
    total_recall=0
    
    for test_number in range(number_of_tests):
        print("Test %d/%d" %(test_number+1, number_of_tests))
        model = compile_model(labels_earlier)

        # transfer learning
        if load_model_weights and model_weights_file:
                model.load_weights(model_weights_file)

        samples_per_epoch = len(train_sentences)
        epochs = number_of_epochs
        batch_size = batch_size
        steps_per_epoch = math.ceil(samples_per_epoch / batch_size)
        checkpoint = ModelCheckpoint(filename_to_save_weigths, monitor='val_acc',save_best_only = True, 
                                     save_weights_only = True)

        for epoch in range(epochs):
            print("Epoch: %d" %(epoch+1))
            model.fit_generator(
                generator(filename = train_file, batch_size = batch_size, check = check_for_generator, 
                          labels2Idx= labels2Idx,tokenize= tokenize), 
                steps_per_epoch= steps_per_epoch, epochs=1,
                validation_data = generator(filename ='/home/jindal/notebooks/jindal/NER/language_model/twitter_emoji_dev.csv', 
                                            batch_size = batch_size, check = check_for_generator, 
                                           labels2Idx = labels2Idx, tokenize = tokenize),
                validation_steps = math.ceil(len(dev_labels) / batch_size),
                callbacks = [checkpoint]
            )

            
#             testset_features = np.zeros((len(dev_sentences), nb_sequence_length, nb_embedding_dims))
#             for i in range(len(dev_sentences)):
#                 testset_features[i] = process_features(dev_sentences[i], nb_sequence_length, nb_embedding_dims)
#             results = model.predict(testset_features)


# #             idx2Label = {0 : "OTHER", 1 : "OFFENSIVE"}
#             predLabels = results.argmax(axis=-1)
#             devLabels = devLabels
#             f1 = f1_score(devLabels, predLabels, average=f1_measure, pos_label=pos_label) # offensive is the major class. So other is minor
#             r = recall_score(devLabels, predLabels, average=f1_measure, pos_label=pos_label)
#             p = precision_score(devLabels, predLabels, average=f1_measure, pos_label=pos_label)
#             a = accuracy_score(devLabels, predLabels)
#             if max_f1 < f1:
#                 print("model saved. F1 is %f" %(f1))
#                 model.save(filename_to_save_weigths)
#                 max_f1 = f1
#                 max_p = p
#                 max_r = r
#                 max_a = a
#             text = "prec: "+ str(p)+" rec: "+str(r) +" f1: "+str(f1) +" acc: "+str(a)+" \n"
#             print("Test-Data: Prec: %.3f, Rec: %.3f, F1: %.3f, Acc: %.3f" % (p, r, f1, a))
#         to_write= "prec: "+ str(max_p)+" rec: "+str(max_r) +" f1: "+str(max_f1) +" acc: "+str(max_a)+" \n"
#         print(to_write)
#         f.write(to_write)
#         total_f1+=max_f1
#         total_prec+=max_p
#         total_acc+=max_a
#         total_recall+=max_r    
#         print("*****************************************************************************")
#     final_text = "avg_prec: " +str(total_prec/number_of_tests)+" total_rec: "+str(total_recall/number_of_tests) +" total_f1: "+str(total_f1/number_of_tests) +" total_acc: "+str(total_acc/number_of_tests)+" \n"
#     print(final_text)
#     f.write(final_text)
#     f.close()




In [16]:
generator = sequential_generator
train_sentences = train_sentences
devLabels = dev_labels
number_of_tests = 1
number_of_epochs = 10
twitteremojis_pretraining_log = '/home/jindal/notebooks/jindal/NER/language_model/results_pretraining_twitteremojis.txt' 
twitteremojis_pretraining_save_weigths='/home/jindal/notebooks/jindal/NER/language_model/model_pretrained_twitteremojis.h5'
batch_size=32*4
twitteremojis_train_file='/home/jindal/notebooks/jindal/NER/language_model/twitter_emoji_train.csv'
tokenize = False
labels2Idx = labels2Id
f1_measure='macro'
pos_label=0
load_model_weights=False
# model_weights_file:'give filepath as str'=None, 
nb_sequence_length = nb_sequence_length
nb_embedding_dims= nb_embedding_dims
check_for_generator=2



In [17]:
len(labels2Idx)

39

In [ ]:
test_model(generator=generator, 
           train_sentences=train_sentences, 
           devLabels=devLabels,
           number_of_tests= number_of_tests,
          number_of_epochs=number_of_epochs,
           filename_to_log=twitteremojis_pretraining_log, 
           labels_earlier = len(labels2Idx),
           filename_to_save_weigths=twitteremojis_pretraining_save_weigths,
           tokenize= tokenize, 
           labels2Idx= labels2Idx,
          batch_size=batch_size,
           train_file=twitteremojis_train_file, 
           f1_measure=f1_measure, 
           pos_label=pos_label, 
           load_model_weights=load_model_weights,
          nb_sequence_length=nb_sequence_length,
           nb_embedding_dims=nb_embedding_dims, 
           check_for_generator= check_for_generator)

Test 1/1
Epoch: 1
Epoch 1/1
4297/4297 [==============================] - 1373s 320ms/step - loss: 2.3836 - acc: 0.2746 - val_loss: 2.2977 - val_acc: 0.3019
Epoch: 2
Epoch 1/1
4297/4297 [==============================] - 1279s 298ms/step - loss: 2.3120 - acc: 0.2929 - val_loss: 2.2709 - val_acc: 0.3092
Epoch: 3
Epoch 1/1
4297/4297 [==============================] - 1261s 294ms/step - loss: 2.2928 - acc: 0.2991 - val_loss: 2.2600 - val_acc: 0.3104
Epoch: 4
Epoch 1/1
4297/4297 [==============================] - 1264s 294ms/step - loss: 2.2818 - acc: 0.3007 - val_loss: 2.2496 - val_acc: 0.3147
Epoch: 5
Epoch 1/1
4297/4297 [==============================] - 1261s 293ms/step - loss: 2.2733 - acc: 0.3039 - val_loss: 2.2492 - val_acc: 0.3147
Epoch: 6
Epoch 1/1
4297/4297 [==============================] - 1265s 294ms/step - loss: 2.2672 - acc: 0.3052 - val_loss: 2.2418 - val_acc: 0.3152
Epoch: 7
Epoch 1/1
4297/4297 [==============================] - 1263s 294ms/step - loss: 2.2626 - acc: 0.3062

# TRANSFER LEARNING HERE


In [19]:
def test_model_tl(generator, 
               train_sentences, 
               devLabels, 
               number_of_tests,
               number_of_epochs,
               filename_to_log, 
               labels2Idx,
               filename_to_save_weigths,
               batch_size, 
               train_file:'filepath for traininig',
               f1_measure:'binary/macro etc', 
               pos_label:'only if binary f1',
               load_model_weights=False,
               model_weights_file:'give filepath as str'=None, 
               tokenize=True,
               nb_sequence_length = nb_sequence_length,
               nb_embedding_dims= nb_embedding_dims, 
               check_for_generator=None,
                ):
    
    f = open(filename_to_log,"w")
    
   
    total_f1=0
    total_prec=0
    total_acc=0
    total_recall=0
    
    for test_number in range(number_of_tests):
        print("Test %d/%d" %(test_number+1, number_of_tests))
        model = compile_model(39)

        # transfer learning
        if load_model_weights and model_weights_file:
                model.load_weights(model_weights_file)
                print("removing top layer")
                model.layers.pop()
                output = Dense(2, activation = 'softmax')(model.layers[-1].output)
                final_model = Model(inputs=model.input, outputs=[output])
                final_model.compile(loss='categorical_crossentropy', optimizer='nadam', metrics=['accuracy'])
#                 for layer in final_model.layers:
#                     print(layer.name)

        samples_per_epoch = len(train_sentences)
        epochs = number_of_epochs
        batch_size = batch_size
        steps_per_epoch = math.ceil(samples_per_epoch / batch_size)
#         checkpoint = ModelCheckpoint(filename_to_save_weigths, monitor='val_acc',save_best_only = True, 
#                                      save_weights_only = True)


        max_f1=0
        max_p=0
        max_r=0
        max_a=0
        for epoch in range(epochs):
            print("Epoch: %d" %(epoch+1))
            final_model.fit_generator(
                generator(filename = train_file, batch_size = batch_size, check = check_for_generator, 
                          labels2Idx= labels2Idx,tokenize= tokenize), 
                steps_per_epoch= steps_per_epoch, epochs=1,
#                 validation_data = generator(filename ='/home/jindal/notebooks/twitter_data/twitter_classes_500k_dev.csv', 
#                                             batch_size = batch_size, check = check_for_generator, 
#                                            labels2Idx = labels2Idx, tokenize = tokenize),
#                 validation_steps = math.ceil(len(dev_labels) / batch_size),
#                 callbacks = [checkpoint]
            )

            testset_features = np.zeros((len(dev_sentences), nb_sequence_length, nb_embedding_dims))
            for i in range(len(dev_sentences)):
                testset_features[i] = process_features(dev_sentences[i], nb_sequence_length, nb_embedding_dims)
            results = final_model.predict(testset_features)


#             idx2Label = {0 : "OTHER", 1 : "OFFENSIVE"}
            predLabels = results.argmax(axis=-1)
            devLabels = devLabels
            f1 = f1_score(devLabels, predLabels, average=f1_measure, pos_label=pos_label) # offensive is the major class. So other is minor
            r = recall_score(devLabels, predLabels, average=f1_measure, pos_label=pos_label)
            p = precision_score(devLabels, predLabels, average=f1_measure, pos_label=pos_label)
            a = accuracy_score(devLabels, predLabels)
            if max_f1 < f1:
                print("model saved. F1 is %f" %(f1))
                final_model.save(filename_to_save_weigths)
                max_f1 = f1
                max_p = p
                max_r = r
                max_a = a
            text = "prec: "+ str(p)+" rec: "+str(r) +" f1: "+str(f1) +" acc: "+str(a)+" \n"
            print("Test-Data: Prec: %.3f, Rec: %.3f, F1: %.3f, Acc: %.3f" % (p, r, f1, a))
        to_write= "prec: "+ str(max_p)+" rec: "+str(max_r) +" f1: "+str(max_f1) +" acc: "+str(max_a)+" \n"
        print(to_write)
        f.write(to_write)
        total_f1+=max_f1
        total_prec+=max_p
        total_acc+=max_a
        total_recall+=max_r    
        print("*****************************************************************************")
    final_text = "avg_prec: " +str(total_prec/number_of_tests)+" total_rec: "+str(total_recall/number_of_tests) +" total_f1: "+str(total_f1/number_of_tests) +" total_acc: "+str(total_acc/number_of_tests)+" \n"
    print(final_text)
    f.write(final_text)
    f.close()




In [20]:
n_labels =2

In [21]:
train_sentences, train_labels, dev_sentences, dev_labels, labels2Idx = train_dev_sentences(filetrain='/home/gwiedemann/notebooks/OffLang/sample_train.txt',
                   filedev='/home/gwiedemann/notebooks/OffLang/sample_dev.txt', check=3)

In [22]:
print(dev_sentences[0])
print(dev_labels[:20])

@FilmElf_yt Diese ganzen leute die du angesprochen hast sind untermenschen ja aber das alte Deutschland und deutsche volk waren ehrenwerte leute und ein stolzes Land
[1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1]


In [24]:
generator = sequential_generator
train_sentences = train_sentences
devLabels = dev_labels
number_of_tests = 5
number_of_epochs = 50
twitteremojis_tl_log = '/home/jindal/notebooks/jindal/NER/language_model/results_tl_emojis.txt' 
twitteremojis_tl_save_weigths='/home/jindal/notebooks/jindal/NER/language_model/classification_model_tl_emojis.h5'
batch_size=32
twitteremojis_tl_train_file='/home/gwiedemann/notebooks/OffLang/sample_train.txt'
f1_measure='binary'
pos_label=1
load_model_weights=True
model_weights_file = '/home/jindal/notebooks/jindal/NER/language_model/model_pretrained_twitteremojis.h5'
nb_sequence_length = nb_sequence_length
nb_embedding_dims= nb_embedding_dims
labels2Idx = labels2Idx
check_for_generator=3

In [25]:
test_model_tl(generator=generator, 
           train_sentences=train_sentences, 
           devLabels=devLabels, 
           number_of_tests= number_of_tests,
           number_of_epochs=number_of_epochs, 
           filename_to_log=twitteremojis_tl_log,
           labels2Idx = labels2Idx,
           filename_to_save_weigths=twitteremojis_tl_save_weigths,
           batch_size=batch_size,
           train_file=twitteremojis_tl_train_file, 
           f1_measure=f1_measure, 
           pos_label=pos_label, 
           load_model_weights=load_model_weights,
           model_weights_file = model_weights_file, 
           nb_sequence_length=nb_sequence_length, 
           nb_embedding_dims=nb_embedding_dims, 
           check_for_generator= check_for_generator)

Test 1/5
removing top layer
Epoch: 1
Epoch 1/1
132/132 [==============================] - 30s 229ms/step - loss: 0.6356 - acc: 0.6771
model saved. F1 is 0.521327
Test-Data: Prec: 0.710, Rec: 0.412, F1: 0.521, Acc: 0.750
Epoch: 2
Epoch 1/1
132/132 [==============================] - 29s 217ms/step - loss: 0.5462 - acc: 0.7275
model saved. F1 is 0.545455
Test-Data: Prec: 0.694, Rec: 0.449, F1: 0.545, Acc: 0.752
Epoch: 3
Epoch 1/1
132/132 [==============================] - 28s 214ms/step - loss: 0.5111 - acc: 0.7431
model saved. F1 is 0.664122
Test-Data: Prec: 0.677, Rec: 0.652, F1: 0.664, Acc: 0.782
Epoch: 4
Epoch 1/1
132/132 [==============================] - 29s 218ms/step - loss: 0.4950 - acc: 0.7517
Test-Data: Prec: 0.689, Rec: 0.581, F1: 0.630, Acc: 0.775
Epoch: 5
Epoch 1/1
132/132 [==============================] - 29s 221ms/step - loss: 0.4688 - acc: 0.7786
Test-Data: Prec: 0.685, Rec: 0.618, F1: 0.650, Acc: 0.780
Epoch: 6
Epoch 1/1
132/132 [==============================] - 32s 24

132/132 [==============================] - 30s 231ms/step - loss: 0.0920 - acc: 0.9645
Test-Data: Prec: 0.675, Rec: 0.764, F1: 0.717, Acc: 0.801
Epoch: 49
Epoch 1/1
132/132 [==============================] - 31s 233ms/step - loss: 0.0826 - acc: 0.9690
Test-Data: Prec: 0.718, Rec: 0.715, F1: 0.717, Acc: 0.813
Epoch: 50
Epoch 1/1
132/132 [==============================] - 31s 231ms/step - loss: 0.0874 - acc: 0.9643
Test-Data: Prec: 0.695, Rec: 0.715, F1: 0.705, Acc: 0.802
prec: 0.7077464788732394 rec: 0.7528089887640449 f1: 0.7295825771324863 acc: 0.8155940594059405 

*****************************************************************************
Test 2/5
removing top layer
Epoch: 1
Epoch 1/1
132/132 [==============================] - 34s 254ms/step - loss: 0.6188 - acc: 0.6858
model saved. F1 is 0.541284
Test-Data: Prec: 0.698, Rec: 0.442, F1: 0.541, Acc: 0.752
Epoch: 2
Epoch 1/1
132/132 [==============================] - 31s 238ms/step - loss: 0.5399 - acc: 0.7296
model saved. F1 is 0.59

132/132 [==============================] - 31s 237ms/step - loss: 0.1019 - acc: 0.9598
Test-Data: Prec: 0.700, Rec: 0.674, F1: 0.687, Acc: 0.797
Epoch: 46
Epoch 1/1
132/132 [==============================] - 31s 238ms/step - loss: 0.0989 - acc: 0.9609
Test-Data: Prec: 0.707, Rec: 0.685, F1: 0.696, Acc: 0.802
Epoch: 47
Epoch 1/1
132/132 [==============================] - 31s 236ms/step - loss: 0.0892 - acc: 0.9664
Test-Data: Prec: 0.697, Rec: 0.697, F1: 0.697, Acc: 0.800
Epoch: 48
Epoch 1/1
132/132 [==============================] - 31s 237ms/step - loss: 0.0891 - acc: 0.9652
Test-Data: Prec: 0.719, Rec: 0.682, F1: 0.700, Acc: 0.807
Epoch: 49
Epoch 1/1
132/132 [==============================] - 31s 236ms/step - loss: 0.0860 - acc: 0.9702
Test-Data: Prec: 0.697, Rec: 0.708, F1: 0.703, Acc: 0.802
Epoch: 50
Epoch 1/1
132/132 [==============================] - 31s 237ms/step - loss: 0.0779 - acc: 0.9702
Test-Data: Prec: 0.699, Rec: 0.670, F1: 0.685, Acc: 0.796
prec: 0.65527950310559 rec: 0.

132/132 [==============================] - 31s 237ms/step - loss: 0.1135 - acc: 0.9562
Test-Data: Prec: 0.706, Rec: 0.674, F1: 0.690, Acc: 0.800
Epoch: 42
Epoch 1/1
132/132 [==============================] - 31s 237ms/step - loss: 0.1071 - acc: 0.9583
Test-Data: Prec: 0.718, Rec: 0.678, F1: 0.697, Acc: 0.806
Epoch: 43
Epoch 1/1
132/132 [==============================] - 31s 233ms/step - loss: 0.1233 - acc: 0.9531
Test-Data: Prec: 0.710, Rec: 0.670, F1: 0.690, Acc: 0.801
Epoch: 44
Epoch 1/1
132/132 [==============================] - 31s 235ms/step - loss: 0.0956 - acc: 0.9626
Test-Data: Prec: 0.711, Rec: 0.674, F1: 0.692, Acc: 0.802
Epoch: 45
Epoch 1/1
132/132 [==============================] - 31s 235ms/step - loss: 0.1029 - acc: 0.9595
Test-Data: Prec: 0.689, Rec: 0.738, F1: 0.712, Acc: 0.803
Epoch: 46
Epoch 1/1
132/132 [==============================] - 31s 234ms/step - loss: 0.1084 - acc: 0.9557
Test-Data: Prec: 0.722, Rec: 0.682, F1: 0.701, Acc: 0.808
Epoch: 47
Epoch 1/1
132/132 [=

132/132 [==============================] - 30s 231ms/step - loss: 0.1380 - acc: 0.9486
Test-Data: Prec: 0.697, Rec: 0.689, F1: 0.693, Acc: 0.798
Epoch: 38
Epoch 1/1
132/132 [==============================] - 31s 232ms/step - loss: 0.1206 - acc: 0.9538
Test-Data: Prec: 0.708, Rec: 0.670, F1: 0.688, Acc: 0.800
Epoch: 39
Epoch 1/1
132/132 [==============================] - 30s 230ms/step - loss: 0.1237 - acc: 0.9527
Test-Data: Prec: 0.723, Rec: 0.655, F1: 0.688, Acc: 0.803
Epoch: 40
Epoch 1/1
132/132 [==============================] - 31s 232ms/step - loss: 0.1103 - acc: 0.9579
Test-Data: Prec: 0.691, Rec: 0.637, F1: 0.663, Acc: 0.786
Epoch: 41
Epoch 1/1
132/132 [==============================] - 31s 234ms/step - loss: 0.1057 - acc: 0.9616
Test-Data: Prec: 0.714, Rec: 0.655, F1: 0.684, Acc: 0.800
Epoch: 42
Epoch 1/1
132/132 [==============================] - 30s 230ms/step - loss: 0.1110 - acc: 0.9595
Test-Data: Prec: 0.697, Rec: 0.682, F1: 0.689, Acc: 0.797
Epoch: 43
Epoch 1/1
132/132 [=

132/132 [==============================] - 31s 238ms/step - loss: 0.1448 - acc: 0.9432
Test-Data: Prec: 0.673, Rec: 0.693, F1: 0.683, Acc: 0.787
Epoch: 35
Epoch 1/1
132/132 [==============================] - 31s 238ms/step - loss: 0.1403 - acc: 0.9441
Test-Data: Prec: 0.701, Rec: 0.640, F1: 0.669, Acc: 0.791
Epoch: 36
Epoch 1/1
132/132 [==============================] - 32s 241ms/step - loss: 0.1421 - acc: 0.9403
Test-Data: Prec: 0.692, Rec: 0.689, F1: 0.690, Acc: 0.796
Epoch: 37
Epoch 1/1
132/132 [==============================] - 31s 235ms/step - loss: 0.1372 - acc: 0.9453
Test-Data: Prec: 0.735, Rec: 0.633, F1: 0.680, Acc: 0.803
Epoch: 38
Epoch 1/1
132/132 [==============================] - 31s 234ms/step - loss: 0.1187 - acc: 0.9541
Test-Data: Prec: 0.704, Rec: 0.659, F1: 0.681, Acc: 0.796
Epoch: 39
Epoch 1/1
132/132 [==============================] - 31s 235ms/step - loss: 0.1140 - acc: 0.9567
Test-Data: Prec: 0.659, Rec: 0.738, F1: 0.696, Acc: 0.787
Epoch: 40
Epoch 1/1
132/132 [=